# Start


Quick solution for tensorflow module not found


In [ ]:
# !pip install tensorflow

In [ ]:
import os
import shutil

import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import seaborn as sns
from PIL import Image
import cv2
import os
from tensorflow.keras.models import Sequential
from PIL import Image
import pandas as pd
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,InputLayer, Rescaling

## Import data


In [ ]:
def import_raw_data():
    # Load dataset
    path = kagglehub.dataset_download("arashnic/faces-age-detection-dataset")

    # Define path to files
    faces_path = os.path.join(path, "faces")
    train_path = os.path.join(faces_path, "Train")

    # Create directory for raw data
    export_path = "../data/raw/faces"
    os.makedirs(export_path, exist_ok=True)

    # Copy files to raw data directory
    shutil.copytree(train_path, os.path.join(export_path, "images"), dirs_exist_ok=True)
    shutil.copy(os.path.join(faces_path, "train.csv"), os.path.join(export_path, "train.csv"))

    export_path = "../data/raw"


if not os.path.exists("../data/raw/faces"):
    import_raw_data()

-   Faces_02 has no csv file for classification so we are gonna skip it


## Definitons


In [ ]:
df_train = pd.read_csv("../data/raw/faces/train.csv")
IMAGE_SIZE = 128
BATCH_SIZE = 32

# 4.1


## A


-   EDA a data preprocessing pre Vami vybrané charakteristiky z datasetu


### EDA


In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train.isnull().sum()

In [ ]:
image_dir = "../data/raw/faces/images"
num_files = len([name for name in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, name))])
print(f"Number of images: {num_files}")

In [ ]:
print(df_train["Class"].value_counts())
df_train["Class"].value_counts().plot(kind="bar")
plt.xticks(rotation=45)
plt.show()

-   There are no nulls
-   There are no duplicates
-   Number of images is same as number of data in cvs file


### Data preprocessing


In [ ]:
# Load your CSV file
image_dir = "../data/raw/faces/images"

def is_low_quality(image_path, min_width=50, min_height=50, threshold=50.0):
    try:
        img = Image.open(image_path)
        img2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img2 is None:
            return True
        laplacian_var = cv2.Laplacian(img2, cv2.CV_64F).var()
        width, height = img.size
        return width < min_width or height < min_height or laplacian_var < threshold
    
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return True  

if (not os.path.exists("../data/processed")):
    low_quality_images = []
    for image_id in df_train['ID']:
        image_path = os.path.join(image_dir, image_id)
        if not os.path.exists(image_path):
            print(f"File not found: {image_path}")
            break
        if is_low_quality(image_path):
            low_quality_images.append(image_id)

    print(f"Number of low quality images: {len(low_quality_images)}")

    os.makedirs("../data/processed", exist_ok=True)
    df_train = df_train[~df_train['ID'].isin(low_quality_images)]
    df_train.to_csv("../data/processed/train.csv", index=False)

In [ ]:
def resize_images(images, target_size):
    output_dir = "../data/processed/faces"

    young_dir = os.path.join(output_dir, "young")
    middle_dir = os.path.join(output_dir, "middle")
    old_dir = os.path.join(output_dir, "old")

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(young_dir, exist_ok=True)
    os.makedirs(middle_dir, exist_ok=True)
    os.makedirs(old_dir, exist_ok=True)

    for _, row in images.iterrows():
        img_name = row["ID"]
        img_path = os.path.join("../data/raw/faces/images", img_name)
        img = image.load_img(img_path, target_size=target_size, color_mode="grayscale")

        if row["Class"].upper() == "YOUNG":
            img.save(os.path.join(young_dir, img_name))
        elif row["Class"].upper() == "MIDDLE":
            img.save(os.path.join(middle_dir, img_name))
        elif row["Class"].upper() == "OLD":
            img.save(os.path.join(old_dir, img_name))


if not os.path.exists("../data/processed/faces"):
    resize_images(df_train, target_size=(IMAGE_SIZE, IMAGE_SIZE))

In [ ]:
ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/processed/faces", shuffle=True, image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE
)

#### Splitting Data


In [ ]:
# Code from: IAU_01_CNN-tf-keras_Lung-cancer.ipynb
def splitDataset(data, train_split=0.7, val_split=0.2, test_split=0.1):
    data_size = len(data)

    train_size = int(train_split * data_size)
    val_size = int(val_split * data_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds


train_ds, val_ds, test_ds = splitDataset(ds)

## B


-   Zdôvodnite výber ML/DL metód vzhľadom na Vami vybraný dataset pre 4.2


# 4.2


## A


-   Modeluje Vami tie vybrané charakteristiky pomocou vhodných ML/DL
    metód. Výsledok modelovania je najlepší model.


In [ ]:
input_shape = None
for batch, labels in train_ds.take(1):
    input_shape = batch.shape

model = Sequential(
    [
        InputLayer(shape=input_shape[1:]),
        Rescaling(1.0 / 255),
        Conv2D(32, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        Conv2D(32, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dropout(0.3),
        Dense(32, activation="relu"),
        Dense(3, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam", loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_ds, epochs=6, validation_data=val_ds, batch_size=BATCH_SIZE, validation_batch_size=BATCH_SIZE
)

In [ ]:
loss, acc = model.evaluate(val_ds, batch_size=BATCH_SIZE)
print(f"Loss: {round(loss, 3)}, Acc: {round(acc*100, 3)}%")

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(range(6), history.history["val_loss"], label="Validation Loss")
plt.plot(range(6), history.history["loss"], label="Loss")
plt.title("Loss")
plt.legend()
plt.show()

## B


-   Zhodnotíte Váš prístup a získaný výsledok


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:")
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)

In [ ]:
import tensorflow as tf

# Method 1: Check if CUDA is available
print("Built with CUDA:", tf.test.is_built_with_cuda())

# Method 2: List physical devices
print("\nPhysical Devices:")
print(tf.config.list_physical_devices())
print("\nGPU Devices:")
print(tf.config.list_physical_devices("GPU"))

# Method 3: Run operation to check device placement
with tf.device("/GPU:0"):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
    print("\nMatrix multiplication result:", c)
    print("Device tensor is on:", c.device)

# Method 4: Get current device
print("\nTensorFlow version:", tf.__version__)
print("Default GPU Device: {}".format(tf.test.gpu_device_name()))